In [2]:
import pandas as pd
from datetime import datetime

In [3]:
locations = [
    'Ho Chi Minh',
    'Ben Tre',
    'DaNang',
    'HaNoi',
    'LamDong',
    'LangSon',
    'NgheAn',
    'SonLa'
]
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_csv(f'Data/{locations[0]}.csv', index_col=0)

for i in range(1, len(locations)):
    tmp = pd.read_csv(f'Data/{locations[i]}.csv', index_col=0)
    df = pd.concat([df, tmp], axis=0).reset_index(drop=True)

df.to_csv('Data/raw_dataset.csv')

In [5]:
df = pd.read_csv('Data/raw_dataset.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14616 entries, 0 to 14615
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Address         14616 non-null  object 
 1   Datetime        14616 non-null  object 
 2   DatetimeEpoch   14616 non-null  int64  
 3   Tempmax         14616 non-null  float64
 4   Tempmin         14616 non-null  float64
 5   Temp            14616 non-null  float64
 6   Dew             14616 non-null  float64
 7   Humidity        14616 non-null  float64
 8   Precip          14616 non-null  float64
 9   Precipprob      14616 non-null  float64
 10  Precipcover     14616 non-null  float64
 11  Preciptype      10414 non-null  object 
 12  Windgust        14616 non-null  float64
 13  Windspeed       14616 non-null  float64
 14  Winddir         14616 non-null  float64
 15  Pressure        14616 non-null  float64
 16  Cloudcover      14616 non-null  float64
 17  Visibility      13504 non-null 

In [6]:
def missing_ratio(x):
    return x.isna().mean()

def fill_mean(x):
    return x.fillna(x.mean())

In [7]:
for location in locations:
    df = pd.read_csv(f'Data/{location}.csv', index_col=0)
    tmp = df.agg([missing_ratio])
    filtered = [h for h in tmp.columns if tmp[h].iloc[0] > 0]
    print(tmp[filtered])

               Preciptype
missing_ratio    0.254516
               Preciptype
missing_ratio    0.238643
               Preciptype
missing_ratio     0.41434
               Preciptype
missing_ratio    0.282978
               Preciptype  Visibility
missing_ratio     0.15873    0.604817
               Preciptype  Visibility
missing_ratio    0.249042    0.000547
               Preciptype  Visibility
missing_ratio    0.343186    0.001642
               Preciptype  Visibility
missing_ratio    0.358511    0.001642


In [8]:
for location in locations:
    df = pd.read_csv(f'Data/{location}.csv', index_col=0)

    df['Visibility'] = df.groupby('Address')['Visibility'].transform(fill_mean)
    tmp = df.agg([missing_ratio])
    filtered = [h for h in tmp.columns if tmp[h].iloc[0] > 0]
    print(tmp[filtered])
    df.to_csv(f'Data/cleaned/{location}.csv')

               Preciptype
missing_ratio    0.254516
               Preciptype
missing_ratio    0.238643
               Preciptype
missing_ratio     0.41434
               Preciptype
missing_ratio    0.282978
               Preciptype
missing_ratio     0.15873
               Preciptype
missing_ratio    0.249042
               Preciptype
missing_ratio    0.343186
               Preciptype
missing_ratio    0.358511


In [9]:
# kiểm tra mâu thuẫn
for location in locations:
    # 1. tempmin <= temp <= tempmax
    df = pd.read_csv(f'Data/{location}.csv', index_col=0)
    tmp = df[['Tempmax', 'Tempmin', 'Temp']].copy()
    for i, row in tmp.iterrows():
        assert (row['Tempmin'] <= row['Temp'] <= row['Tempmax'])

    # 2. datetime và datetimeepoch
    def sec(date2):
        date1 = "1970-01-01"
        date_format = "%Y-%m-%d"
        tzoffset = 7
        datetime1 = datetime.strptime(date1, date_format)
        datetime2 = datetime.strptime(date2, date_format)

        return (datetime2 - datetime1).total_seconds() - tzoffset * 3600

    tmp = df[['Datetime', 'DatetimeEpoch']].copy()
    for i, row in tmp.iterrows():
        assert row['DatetimeEpoch'] == sec(row['Datetime'])

    # 3. Kiểm tra duplicate
    assert df.duplicated().sum() == 0